# Customers and Products Analysis Using SQL

In [91]:
# import libraries

import sqlite3 as sql

In [92]:
#Establishing Connection
conn = sql.connect('/Users/taranjitkaur/downloads/stores.db')
df = pd.read_sql_query("""SELECT * 
                            FROM customers;""", conn)
df

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,None,Philadelphia,PA,71270,USA,1323.0,72600
118,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,None,Brisbane,CA,94217,USA,1165.0,60300
119,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,None,London,None,WA1 1DP,UK,1501.0,43300
120,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,None,Boston,MA,51003,USA,1188.0,85100


In [80]:
#Pragma Table Info
>>> conn = sql.connect(':mem:')
>>> for row in conn.execute("pragma table_info('sqlite_master')").fetchall():
    print (row)

(0, 'type', 'text', 0, None, 0)
(1, 'name', 'text', 0, None, 0)
(2, 'tbl_name', 'text', 0, None, 0)
(3, 'rootpage', 'int', 0, None, 0)
(4, 'sql', 'text', 0, None, 0)


In [81]:
%%capture
%load_ext sql
%sql sqlite:////Users/taranjitkaur/downloads/stores.db

In [82]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");


 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


name,type
customers,table
employees,table
offices,table
orderdetails,table
orders,table
payments,table
productlines,table
products,table
profit_report,view
profit,view


# Exploring Database
Writing a query to display the following table:

Select each table name as a string.
Select the number of attributes as an integer (count the number of attributes per table).
Select the number of rows using the COUNT(*) function.
Use the compound-operator UNION ALL to bind these rows together.


In [83]:
%%sql
SELECT '' AS Table_Names,'' AS Number_of_Attributes,'' AS Number_of_Rows
UNION ALL
SELECT 'Customer',13,(SELECT COUNT(*)
                        FROM customers)
UNION ALL
SELECT 'Products',9,(SELECT COUNT(*)
                        FROM products)
UNION ALL
SELECT 'ProductLines',4,(SELECT COUNT(*)
                        FROM productlines)
UNION ALL
SELECT 'Orders',7,(SELECT COUNT(*)
                        FROM orders)
UNION ALL
SELECT 'OrderDetails',5,(SELECT COUNT(*)
                        FROM orderdetails)
UNION ALL
SELECT 'Payments',4,(SELECT COUNT(*)
                        FROM payments)
UNION ALL
SELECT 'Employees',8,(SELECT COUNT(*)
                        FROM employees)
UNION ALL
SELECT 'Offices',9,(SELECT COUNT(*)
                        FROM offices)

 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


Table_Names,Number_of_Attributes,Number_of_Rows
,,
Customer,13,122
Products,9,110
ProductLines,4,7
Orders,7,326
OrderDetails,5,2996
Payments,4,273
Employees,8,23
Offices,9,7


# Which one is the best selling?: 
For to do this, I had to group the sales by product code using the syntax below:

In [84]:
%%sql
SELECT 
    productCode, 
ROUND(SUM( quantityOrdered* priceEach),2)AS Total_sales
FROM orderdetails
GROUP BY productCode
ORDER BY Total_sales DESC
LIMIT 10;


 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


productCode,Total_sales
S18_3232,276839.98
S12_1108,190755.86
S10_1949,190017.96
S10_4698,170686.0
S12_1099,161531.48
S12_3891,152543.02
S18_1662,144959.91
S18_2238,142530.63
S18_1749,140535.6
S12_2823,135767.03


### 1992 Ferrari 360 Spider red has the best performance 

# Display priority products for restocking

In [85]:
%%sql
WITH
--#Ten most popular products
get_lowstock AS(
SELECT productCode,
       --#Use low stock formula
       (SUM(od.quantityOrdered)*1.0)/ 
                       (SELECT quantityInStock
                          FROM products
                         WHERE od.productCode = productCode) AS low_stock
 FROM orderdetails od
GROUP BY productCode
ORDER BY low_stock DESC
LIMIT 10
)
SELECT productName,
       quantityInStock ,
       COUNT(*) AS No_of_Orders,
       SUM(quantityOrdered) AS Total_Quantity_Ordered,
       --#Use product performance formula
       ROUND(SUM(quantityOrdered*priceEach),2) AS Total_Sales,
       products.productLine AS Product_Line
  FROM orderdetails
  JOIN products
    ON products.productCode = orderdetails.productCode
 WHERE orderdetails.productCode IN (SELECT productCode
                                               FROM get_lowstock)
 GROUP BY products.productCode
 ORDER BY Total_Sales DESC
 --LIMIT 10;

 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


productName,quantityInStock,No_of_Orders,Total_Quantity_Ordered,Total_Sales,Product_Line
1968 Ford Mustang,68,27,933,161531.48,Classic Cars
1928 Mercedes-Benz SSK,548,28,880,132275.98,Vintage Cars
1997 BMW F650 ST,178,28,1014,89364.89,Motorcycles
F/A 18 Hornet 1/72,551,28,1047,76618.4,Planes
2002 Yamaha YZR M1,600,27,992,73670.64,Motorcycles
The Mayflower,737,27,898,69531.61,Ships
1960 BSA Gold Star DBD34,15,28,1015,67193.49,Motorcycles
1928 Ford Phaeton Deluxe,136,28,972,60493.33,Vintage Cars
Pont Yacht,414,27,958,47550.4,Ships
1911 Ford Town Car,540,25,832,45306.77,Vintage Cars


# Which country sold the most product?

Using the query syntax below, the data revealed that Spain sold the most product out of the 122 other countries.


In [86]:
%%sql
DROP VIEW net_profit;
print()

 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.
(sqlite3.OperationalError) near "print": syntax error
[SQL: print()]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [87]:
%%sql
CREATE VIEW net_profit AS
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber;  

SELECT customers.city AS City,
       customers.country AS Country,
       net_profit.profit AS Profit_Earned
  FROM customers
  JOIN net_profit
    ON customers.customerNumber = net_profit.customerNumber
 WHERE net_profit.customerNumber IN (SELECT customerNumber
                                                FROM net_profit
                                               ORDER BY profit DESC
                                               LIMIT 5)
 ORDER BY net_profit.profit DESC;

 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.
Done.


City,Country,Profit_Earned
Madrid,Spain,326519.65999999986
San Rafael,USA,236769.38999999998
NYC,USA,72370.09000000001
Melbourne,Australia,70311.06999999999
Nantes,France,60875.30000000001


# Profits earned by Salesman

In [89]:
%%sql
--#Join customers, employees and offices
SELECT employees.lastName||', '||employees.firstName AS Sales_Rep_Name,
       offices.territory,
       offices.city,
       ROUND(SUM(net_profit.profit),2) AS Profit_Earned,
       ROUND((SUM(net_profit.profit) * 100.0 / 
                          (SELECT SUM(profit)
                             FROM net_profit)), 2) AS Percentage
   FROM customers
   JOIN net_profit
     ON customers.customerNumber = net_profit.customerNumber
   JOIN employees
     ON customers.salesRepEmployeeNumber = employees.employeeNumber
   JOIN offices
     ON employees.officeCode=offices.officeCode
  GROUP BY employees.employeeNumber
  ORDER BY Profit_Earned DESC;

 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


Sales_Rep_Name,territory,city,Profit_Earned,Percentage
"Hernandez, Gerard",EMEA,Paris,504644.71,13.19
"Jennings, Leslie",NA,San Francisco,435208.35,11.38
"Castillo, Pamela",EMEA,Paris,340727.9,8.91
"Bott, Larry",EMEA,London,290203.59,7.59
"Jones, Barry",EMEA,London,276659.25,7.23
"Vanauf, George",NA,NYC,269596.09,7.05
"Bondur, Loui",EMEA,Paris,234891.07,6.14
"Marsh, Peter",APAC,Sydney,230811.75,6.03
"Fixter, Andy",APAC,Sydney,222207.18,5.81
"Patterson, Steve",NA,Boston,197879.23,5.17


# Best Month for Sales

In [90]:
%%sql 
SELECT CAST(SUBSTR(paymentDate,1,4) AS INT)*100+CAST(SUBSTR(paymentDate,6,7) AS INT) AS Year_Month,
       CAST(SUBSTR(paymentDate,1,4) AS INT) AS Year,
       CAST(SUBSTR(paymentDate,6,7) AS INT) AS Month,
       ROUND(SUM(amount),2) AS Amount
  FROM payments
 GROUP BY Year_Month
HAVING Year_Month < 200501
LIMIT 10;


 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


Year_Month,Year,Month,Amount
200301,2003,1,26267.62
200302,2003,2,144384.36
200303,2003,3,199704.48
200304,2003,4,136313.92
200305,2003,5,159881.97
200306,2003,6,180218.98
200307,2003,7,158247.0
200308,2003,8,246204.86
200309,2003,9,161206.23
200310,2003,10,316857.96


# How Much Can We Spend on Acquiring New Customers?

In [71]:
%%sql
WITH 

payment_with_year_month_table AS (
SELECT *, 
       CAST(SUBSTR(paymentDate, 1,4) AS INTEGER)*100 + CAST(SUBSTR(paymentDate, 6,7) AS INTEGER) AS year_month
  FROM payments p
),

customers_by_month_table AS (
SELECT p1.year_month, COUNT(*) AS number_of_customers, SUM(p1.amount) AS total
  FROM payment_with_year_month_table p1
 GROUP BY p1.year_month
),

new_customers_by_month_table AS (
SELECT p1.year_month, 
       COUNT(*) AS number_of_new_customers,
       SUM(p1.amount) AS new_customer_total,
       (SELECT number_of_customers
          FROM customers_by_month_table c
        WHERE c.year_month = p1.year_month) AS number_of_customers,
       (SELECT total
          FROM customers_by_month_table c
         WHERE c.year_month = p1.year_month) AS total
  FROM payment_with_year_month_table p1
 WHERE p1.customerNumber NOT IN (SELECT customerNumber
                                   FROM payment_with_year_month_table p2
                                  WHERE p2.year_month < p1.year_month)
 GROUP BY p1.year_month
)

SELECT year_month, 
       ROUND(number_of_new_customers*100/number_of_customers,1) AS number_of_new_customers_props,
       ROUND(new_customer_total*100/total,1) AS new_customers_total_props
  FROM new_customers_by_month_table;

 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


year_month,number_of_new_customers_props,new_customers_total_props
200301,100.0,100.0
200302,100.0,100.0
200303,100.0,100.0
200304,100.0,100.0
200305,100.0,100.0
200306,100.0,100.0
200307,75.0,68.3
200308,66.0,54.2
200309,80.0,95.9
200310,69.0,69.3


# The best and worst product lines in different countries

In [94]:
%%sql
WITH
--#Join products and orderdetails 
order_category AS(
SELECT orderdetails.orderNumber,
       products.productLine
  FROM products
  JOIN orderdetails
    ON products.productCode=orderdetails.productCode
),

--#Join customers and orders
country_order AS(
SELECT customers.country,
       orders.orderNumber
  FROM orders
  JOIN customers
    ON orders.customerNumber=customers.customerNumber
),

--#Combine the above joins
country_category AS(
SELECT country_order.country AS Country,
       order_category.productLine AS Category
  FROM country_order
  JOIN order_category
    ON country_order.orderNumber=order_category.orderNumber
),

--#List all categories and their counts along with each country
country_category_count AS(
SELECT Country,
       Category,
       COUNT(Category) AS category_count
  FROM country_category
 GROUP BY Country,Category
)

--#Select the category with the most and least count
SELECT Country,
       (SELECT Category
          FROM country_category_count
         GROUP BY Country
        HAVING Country=c.Country AND category_count=MAX(category_count)) AS Best_Category,
       (SELECT Category
          FROM country_category_count
         GROUP BY Country
        HAVING Country=c.Country AND category_count=MIN(category_count)) AS Worst_Category
  FROM country_category_count c
 GROUP BY Country

 * sqlite:////Users/taranjitkaur/downloads/stores.db
Done.


Country,Best_Category,Worst_Category
Australia,Vintage Cars,Trains
Austria,Classic Cars,Ships
Belgium,Vintage Cars,Planes
Canada,Trucks and Buses,Motorcycles
Denmark,Classic Cars,Planes
Finland,Classic Cars,Trains
France,Classic Cars,Trains
Germany,Classic Cars,Ships
Hong Kong,Planes,Motorcycles
Ireland,Classic Cars,Trucks and Buses
